In [ ]:
Pkg.add("PyPlot")
Pkg.add("DataStructures")
Pkg.update()

In [1]:
include("kernel.jl")

getInversion (generic function with 1 method)

# InitialIze KIFMM kernel 

The aim of this notebook is to illustrate usage of ``KIFMM`` to calculate ``p = K * q``.

necessary notation:

- ``k``: kernel, a struct to hold everything
- ``source``: source points in 2D
- ``normal``: normal vectors(optional) at each source points in 2D
- ``target``: target points in 2D

- ``f``: self defined kernel function, signature is

    ```
    function f(source, normal, target)
        ...
        return someNumber
    end
    ```

    example usage:
    ```
    output = f([0.5, 0.2], [0.6, 0.8], [0.7, 0.4])
    ```
    
### Calling ``initialize! `` 

When to initialize the kernel, one can call ``initialize!``.

```
initialize!(kernel, np, source, normal, target, charge, N, M, rank, maxLevel, f)
```

where

- ``np`` is the number of surface points to approximate boundary integral(potential theory).
- ``charge`` is the vector ``q`` to be applied.
- ``N``, number of source
- ``M``, number of target
- ``rank``, max source points in each leaf box
- ``maxLevel``, max level of tree


In [2]:
k = KIFMM()
N = 9000
source = rand(N, 2)
target = rand(N, 2)
charge = rand(N)

### kernel function for Laplacian, this kernel does not require normal vectors
function f(s, n, t)
    r =  norm(s-t)
    if (r < 1e-4)
        return 0.
    else
        return log(norm(s-t))
    end
end

tic()
initialize!(k, 16, source, source, target, charge, N, N, 40, 10, f);
toc()

(0.5177, 0.0001) is outside of source box.
(0.4192, 0.0001) is outside of source box.
(0.0001, 0.0707) is outside of source box.
(0.0000, 0.6179) is outside of source box.
(0.4192, 0.0001) is outside of source box.
(0.0001, 0.0707) is outside of source box.
(0.5177, 0.0001) is outside of source box.
(0.0000, 0.6179) is outside of source box.
(0.0001, 0.0707) is outside of source box.
(0.4192, 0.0001) is outside of source box.
(0.0001, 0.0707) is outside of source box.
(0.0001, 0.0707) is outside of source box.
(0.4192, 0.0001) is outside of source box.
(0.4192, 0.0001) is outside of source box.
(0.5177, 0.0001) is outside of source box.
(0.5177, 0.0001) is outside of source box.
(0.5177, 0.0001) is outside of source box.
(0.0000, 0.6179) is outside of source box.
(0.0000, 0.6179) is outside of source box.
(0.0000, 0.6179) is outside of source box.
(0.0000, 0.6179) is outside of source box.
elapsed time: 1.676239151 seconds


1.676239151

### display the tree

To display the tree and all interaction lists (UList, VList, WList, XList), use
```
display(tree, [index of nodes])
```

- black node is the central node
- red is Ulist
- blue is Wlist
- green is Vlist
- grey is Xlist

In [ ]:
display(k.tree, [89, 180])

### FMM!
simply call 
```
p = FMM!(kernel)
```

to get the result ``p = K * q`` using ``KIFMM``

In [ ]:
tic()
potential = FMM!(k)
toc()

### check result using pure interaction (without any matrix operations)

In [ ]:
tic()
checkPotential = zeros(N)
for i = 1:N
    for j = 1:N
        checkPotential[i] += k.kernel(k.tree.sourceTree[j,:], k.tree.normalTree[j,:], k.tree.targetTree[i,:]) * k.chargeTree[j]
    end
end
toc()
norm(potential - checkPotential)/norm(checkPotential)